In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

data_path = 'data/'

lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb27 = pd.read_csv('../lgCV_2705B.csv.gz', compression='gzip')
rnn = pd.read_csv('../rnnCV_2805.csv.gz', compression='gzip')
mlp = pd.read_csv('../mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv('../../%strain.csv.zip'%(data_path), compression='zip')
y = truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv('../../%stest.csv.zip'%(data_path), compression='zip')
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [2]:
lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb25.merge(rnn, on='item_id')\
                .merge(lgb27, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [3]:
preds_df['difference_l27_r']=preds_df['lgb27_preds'] - preds_df['rnn_preds']
preds_df['difference_l27_m']=preds_df['lgb27_preds'] - preds_df['mlp_preds']
preds_df['difference_l27_l25']=preds_df['lgb27_preds'] - preds_df['lgb25_preds']
preds_df['price'].fillna(-1,inplace=True)

In [4]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if '_preds' in c]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb25_preds:  0.2167875440794737
RMSE rnn_preds:  0.21771476573780568
RMSE lgb27_preds:  0.21681527582821059
RMSE mlp_preds:  0.2187544351068867


In [5]:
cols = ['lgb27_preds', 'lgb25_preds', 'rnn_preds','mlp_preds','price', \
        'difference_l27_r', 'difference_l27_m', 'difference_l27_l25',\
        'region', 'param_1', 'parent_category_name', 'category_name']
for col in ['region', 'param_1', 'parent_category_name', 'category_name']:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [6]:
full=False
if full:
    n_estimators = 950
    train_X = train_df[cols]
    train_y = y
    eval_set = [(train_X,train_y)]
else:
    n_estimators = 4000
    train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
    eval_set = [(train_X,train_y),(valid_X,valid_y)]

/home/darragh/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
len(train_X)

1202739

In [8]:
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.5,
                    num_leaves=32, 
                    learning_rate=.02)#, device='gpu')

In [ ]:
%%time
clf.fit(train_X, train_y, early_stopping_rounds=40, eval_set=eval_set, eval_metric='rmse', verbose=10, categorical_feature=['param_1','region','parent_category_name','category_name'])

/home/darragh/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['category_name', 'param_1', 'parent_category_name', 'region']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 40 rounds.
[10]	training's rmse: 0.246134	valid_1's rmse: 0.246286
[20]	training's rmse: 0.236311	valid_1's rmse: 0.236503
[30]	training's rmse: 0.229703	valid_1's rmse: 0.229937
[40]	training's rmse: 0.224821	valid_1's rmse: 0.225104


In [ ]:
test_df['deal_probability'] = clf.predict(test_df[cols] )
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_3105L2.csv.gz', compression='gzip', index=False, header=True)

In [ ]:
test_df[['item_id', 'deal_probability']].head()